# DOCUMENTO DE PRUEBA GOOGLE AI

OBJETIVOS :
- Lograr una respuesta preliminar con el modelo.
- Integrarle un prompt y memoria.
- integrar el LLM al modulo del proyecto.

DOCUMENTACION :

https://python.langchain.com/docs/integrations/llms/google_ai

***OBJETIVO NUMERO 1***

In [1]:
from langchain_google_genai import GoogleGenerativeAI

In [2]:
from getpass import getpass
api_key = getpass()

In [3]:
#EN ESTE CASO SOLO USAMOS EL LLM PARA VERIFICAR UNA RESPUESTA RAPIDA
llm = GoogleGenerativeAI(model="gemini-1.0-pro", google_api_key=api_key)
print(
    llm.invoke(
        "hola chat, como estas!"
    )
)

Hola, estoy bien, gracias por preguntar. ¿Cómo estás tú?


**OBJETIVO NUMERO 2**

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [14]:
# RESPUESTA PRELIMINAR CON PROMPT DEFINIDO
llm = GoogleGenerativeAI(model="gemini-1.0-pro", google_api_key=api_key)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', return_messages=True, output_key='text')
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [19]:
respuesta = chain({'question':"Genial, que es un LLM Open source?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
[HumanMessage(content='hola chat'), AIMessage(content='Hola, ¿cómo estás?'), HumanMessage(content='muy bien, me gustaria saber que es un LLM'), AIMessage(content='Un LLM (Large Language Model) es un tipo de inteligencia artificial (IA) que se utiliza para procesar y generar texto. Los LLM están entrenados en conjuntos de datos masivos de texto y pueden comprender y producir lenguaje humano de forma similar a la de un hablante nativo.\n\nAlgunos de los usos más comunes de los LLM incluyen:\n\n* **Generación de texto:** Los LLM se pueden utilizar para generar texto nuevo, como artículos, historias, poemas y diálogos.\n* **Traducción:** Los LLM se utilizan para traducir texto de un idioma a otro.\n* **Resumen:** Los LLM se pueden utilizar para resumir textos largos en resúmenes más cortos y fáciles de leer.\n* **Clasificación:** Los LLM se pueden u

In [20]:
respuesta['text']

'Un LLM (Large Language Model) de código abierto es un tipo de IA (Inteligencia Artificial) que está disponible gratuitamente para que cualquiera pueda usarlo y modificarlo. Esto significa que los investigadores y desarrolladores pueden acceder al código subyacente del LLM y ajustarlo para sus propias necesidades específicas.\n\nAlgunos de los LLM de código abierto más populares incluyen:\n\n* GPT-3 de OpenAI\n* BLOOM de BigScience\n* OPT de Meta AI\n\nLos LLM de código abierto tienen varias ventajas sobre los LLM propietarios:\n\n* **Transparencia:** El código subyacente de los LLM de código abierto está disponible para que todos lo inspeccionen, lo que permite a los investigadores y desarrolladores comprender mejor cómo funcionan y tomar decisiones informadas sobre su uso.\n* **Personalización:** Los LLM de código abierto se pueden personalizar para satisfacer necesidades específicas, como entrenar el modelo en un conjunto de datos particular o ajustar la arquitectura del modelo.\n* 

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [23]:
# RESPUESTA PRELIMINAR CON PROMPT PERSONALIZADO
llm = GoogleGenerativeAI(model="gemini-1.0-pro", google_api_key=api_key)

template = """Sos un asesor AI dispuesto a asistir a los usuarios en diferentes tareas y proyectos que ellos tengan..

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history",input_key='question', return_messages=True, output_key="text")
chain = LLMChain(llm=llm,prompt=prompt,verbose=True,memory=memory)

In [24]:
respuesta = chain.invoke({'question':'Hola chat! como esta la situacion del dengue en argentina?'})



> Entering new LLMChain chain...
Prompt after formatting:
Sos un asesor AI dispuesto a asistir a los usuarios en diferentes tareas y proyectos que ellos tengan..

Previous conversation:
[]

New human question: Hola chat! como esta la situacion del dengue en argentina?
Response:

> Finished chain.


In [25]:
respuesta['text']

'Hola, según los últimos datos disponibles, la situación del dengue en Argentina es la siguiente:\n\n* **Casos confirmados:** En 2022, se notificaron 9.500 casos confirmados de dengue en el país.\n* **Provincias afectadas:** Las provincias más afectadas fueron Salta, Chaco, Jujuy, Tucumán y Formosa.\n* **Fallecimientos:** Hasta el momento, se han registrado 6 fallecimientos por dengue en Argentina en 2022.\n* **Situación actual:** La transmisión del virus del dengue continúa activa en algunas zonas del país, especialmente en el norte.\n\n**Recomendaciones:**\n\nPara prevenir el dengue, es importante tomar medidas como:\n\n* Eliminar posibles criaderos de mosquitos (agua estancada en recipientes, neumáticos, etc.).\n* Usar repelente de insectos.\n* Usar ropa que cubra la mayor parte del cuerpo.\n* Colocar mosquiteros en ventanas y puertas.\n* Consultar al médico ante síntomas como fiebre alta, dolor de cabeza intenso y dolor muscular.\n\n**Fuentes:**\n\n* Ministerio de Salud de la Nació

***OBJETIVO NUMERO 3***

Esquema de Asesor del proyecto..

In [3]:
#AGREGANDO HERRAMIENTA DE BUSQUEDA PARA EL AGENTE
import os
os.environ["SERPER_API_KEY"] = "8b139823f4e8976b14208e65520107b9ba3607fa"

In [221]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents.agent_types import AgentType
from langchain.prompts import MessagesPlaceholder

def CrearAsesor():
    #TEMPLATE PROXIMAMENTE MODIFICABLE
        template = """
    Vas a asesor a los usuarios de la mejor manera, recordando sus nombres para ser mas amigable, si no sabes la respuesta no la inventes.
    
    Herramientas: {tools}, {tool_names}
    Previous conversation:
    {chat_history}
    
    New human question: {input}
    {agent_scratchpad}
    Response:"""


        prompt = PromptTemplate.from_template(template)

        search = GoogleSerperAPIWrapper()

        search_tool = Tool(
            name="search_tool",
            description="Herramienta util para buscar informacion actualizada en internet.",
            func=search.run,
            )
        
        tools = [search_tool]
        memory = ConversationBufferMemory(memory_key="chat_history", input_key='input', return_messages=True, output_key = 'output')
        
        llm = GoogleGenerativeAI(model="gemini-1.0-pro", google_api_key=api_key)
        
        #chain = LLMChain(llm=llm,prompt=prompt,verbose=True,memory=memory)
        chat_history = MessagesPlaceholder(variable_name="chat_history")
        
#USAMOS INITIALIZE AGENT YA QUE PERMITE ESTRUCTURAR EL OUTPUT DEL AGENTE
        agent = initialize_agent(tools,
                         llm,
                         agent="conversational-react-description",
                         verbose=True, agent_kwargs={
                        "prompt":[prompt],
                        "memory_prompts": [chat_history],
                        "input_variables": ["input"]},
                        memory = memory,
                        handle_parsing_errors=True
                        )

        return agent

In [222]:
asesor_personal = CrearAsesor()

In [225]:
#LUEGO DE AGREGARLE EL PROMPT Y LA MEMORIA A AGENT_KWARGS, SE LOGRO QUE LLM DE GEMINI RECUERDE CONVERSACIONES.
respuesta = asesor_personal.invoke({'input': 'Claro, ese concepto no entendia muy bien, perfecto, te pregunto, recordas mi nombre?'})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Sí, recuerdo tu nombre, Nicolás.

> Finished chain.


In [226]:
respuesta['output']

'Sí, recuerdo tu nombre, Nicolás.'

In [227]:
respuesta['chat_history']

[HumanMessage(content='hola chat, mi nombre es nicolas un gusto, quiero saber que es trading y a que se refieren con invertir en corto o en largo'),
 AIMessage(content='Hola Nicolás, encantado de conocerte. En pocas palabras, el trading es el acto de comprar y vender activos financieros, como acciones, bonos o divisas, con el objetivo de obtener ganancias. Invertir en corto significa apostar a que el precio de un activo caerá, mientras que invertir en largo significa apostar a que el precio de un activo subirá.'),
 HumanMessage(content='Claro, ese concepto no entendia muy bien, perfecto, te pregunto, recordas mi nombre?'),
 AIMessage(content='Sí, recuerdo tu nombre, Nicolás.')]